In [1]:
%matplotlib inline
# Importing libraries
import matplotlib.pyplot as plt
import pandas as pd

FILEPATH_PREFIX = '../../../book_crawler/data'
SPIDERNAME = 'blackwells'
FILENAME = '2019-05-12T14-51-07.csv'
FILEPATH = '{}/{}/{}'.format(FILEPATH_PREFIX, SPIDERNAME, FILENAME)
FILEPATH

'../../../book_crawler/data/blackwells/2019-05-12T14-51-07.csv'

In [2]:
df = pd.read_csv(FILEPATH)

## Data analisys
Some preliminary analisys of the dataset

In [3]:
print("First 5 rows")
print("------------")
df.head()

First 5 rows
------------


,url,referer_url,src
0,https://blackwells.co.uk/bookshop/basket,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e..."
1,https://blackwells.co.uk/bookshop/search/,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e..."
2,https://blackwells.co.uk/bookshop/home,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e..."
3,https://blackwells.co.uk/bookshop/product/9781...,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e..."
4,https://blackwells.co.uk/bookshop/mapping,https://blackwells.co.uk/bookshop/basket,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" c..."


In [4]:
print("No. of rows and columns")
print("-----------------------")
df.shape

No. of rows and columns
-----------------------


(10919, 3)

In [5]:
print("Check null values")
print("-----------------")
df.isnull().any().any()

Check null values
-----------------


False

In [6]:
print("Check duplicate values")
print("----------------------")
len(df['url'].unique()) != df.shape[0]

Check duplicate values
----------------------


False

In [7]:
print("DataFrame column types")
print("----------------------")
df.info()

DataFrame column types
----------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10919 entries, 0 to 10918
Data columns (total 3 columns):
url            10919 non-null object
referer_url    10919 non-null object
src            10919 non-null object
dtypes: object(3)
memory usage: 256.0+ KB


In [8]:
print("Some stats")
print("----------------")
df.describe()

Some stats
----------------


,url,referer_url,src
count,10919,10919,10919
unique,10919,6375,10525
top,https://blackwells.co.uk/bookshop/product/Geot...,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e..."
freq,1,12,7


## Data cleaning
During the crawling phase (using Scrapy), we restrained the crawled pages to be only the web pages matching with `https://blackwells.co.uk/` domain.

In [9]:
import re

In [10]:
blackwellsRegex = r'^https\:\/\/blackwells\.co\.uk\/'

In [11]:
otherDomainsDataFrame = df[- df['url'].str.match(blackwellsRegex)]

In [12]:
otherDomainsDataFrame.head()

,url,referer_url,src


So Scrapy effectively crawled only pages from the `www.blackwells.co.uk` domain

## Add `shingle_vector` label
Compute shingle vector for each page of the dataframe

In [13]:
#add top level folder to sys.path
import sys
sys.path.append('../../../')

from foxlink_clustering.clustering.shingler import compute_shingle_vector

In [14]:
src = df.iloc[0]['src']

The default value for window size in Foxlink is 3

In [15]:
DEFAULT_WINDOW_SIZE = 3
result = compute_shingle_vector(src, DEFAULT_WINDOW_SIZE)

In [16]:
result

(0, 1, 5, 1, 1, 6, 3, 1)

Set `shingle_vector` label for each row    

In [18]:
df['shingle_vector'] = df.apply(lambda x: compute_shingle_vector(x['src'], DEFAULT_WINDOW_SIZE), axis=1)

In [19]:
df.head()

,url,referer_url,src,shingle_vector
0,https://blackwells.co.uk/bookshop/basket,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 5, 1, 1, 6, 3, 1)"
1,https://blackwells.co.uk/bookshop/search/,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 5, 1, 1, 0, 3, 0)"
2,https://blackwells.co.uk/bookshop/home,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 0, 1, 0, 0, 3, 1)"
3,https://blackwells.co.uk/bookshop/product/9781...,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 1, 1, 1, 0, 0, 1)"
4,https://blackwells.co.uk/bookshop/mapping,https://blackwells.co.uk/bookshop/basket,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" c...","(2, 22, 1, 1, 7, 15, 7, 5)"


## Write dataset to a file

In [20]:
df.to_csv('../../../datasets/blackwells.csv', encoding='utf-8', index=False)

In [21]:
test = pd.read_csv('../../../datasets/blackwells.csv')

In [22]:
test.head()

,url,referer_url,src,shingle_vector
0,https://blackwells.co.uk/bookshop/basket,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 5, 1, 1, 6, 3, 1)"
1,https://blackwells.co.uk/bookshop/search/,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 5, 1, 1, 0, 3, 0)"
2,https://blackwells.co.uk/bookshop/home,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 0, 1, 0, 0, 3, 1)"
3,https://blackwells.co.uk/bookshop/product/9781...,https://blackwells.co.uk/bookshop/home,"\n\n\n \n<!DOCTYPE html>\n<html lang=""e...","(0, 1, 1, 1, 1, 0, 0, 1)"
4,https://blackwells.co.uk/bookshop/mapping,https://blackwells.co.uk/bookshop/basket,"\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=""en"" c...","(2, 22, 1, 1, 7, 15, 7, 5)"


In [23]:
test.shape

(10919, 4)